# 卷积神经网络

In [1]:
%matplotlib inline

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

F:\Anaconda\envs\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets(
    'MNIST_data', one_hot=True, reshape=False)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    conv1_weight = tf.get_variable(
        'c1_w', [5, 5, 1, 8], dtype=tf.float32)
    conv1_bias = tf.get_variable(
        'c1_b', [8], dtype=tf.float32)
    conv1_ = tf.nn.conv2d(
        inputs, conv1_weight, [1, 1, 1, 1], 'VALID')
    # shape=[None, 24, 24, 8]
    conv1 = tf.nn.relu(conv1_ + conv1_bias)
    
    # shape=[None, 12, 12, 8]
    pool1 = tf.nn.max_pool(
        conv1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    conv2_weight = tf.get_variable(
        'c2_w', shape=[5, 5, 8, 16], dtype=tf.float32)
    conv2_bias = tf.get_variable(
        'c2_b', shape=[16], dtype=tf.float32)
    conv2_ = tf.nn.conv2d(
        pool1, conv2_weight, [1, 1, 1, 1], 'VALID')
    # shape=[None, 8, 8, 16]
    conv2 = tf.nn.relu(conv2_ + conv2_bias)
    
    # shape=[None, 4, 4, 16]
    pool2 = tf.nn.max_pool(
        conv2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    tmp = tf.reshape(pool2, [-1, 4 * 4 * 16])
    
    logits = tf.keras.layers.Dense(
        10, activation=None)(tmp)
    output = tf.nn.softmax(logits)
    
    saver = tf.train.Saver()
    
    # 代价函数
    loss = tf.reduce_mean(-1 * tf.reduce_sum(
        labels * tf.log(output + 1e-7),
        axis=1))
    
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 2.3574, acc 0.0920
step   500, loss 0.2700, acc 0.8445
step  1000, loss 0.2615, acc 0.9061
step  1500, loss 0.3365, acc 0.9308
step  2000, loss 0.4377, acc 0.9377
step  2500, loss 0.1906, acc 0.9522
step  3000, loss 0.1526, acc 0.9560
step  3500, loss 0.1710, acc 0.9592
step  4000, loss 0.2550, acc 0.9637
step  4500, loss 0.0118, acc 0.9700
step  5000, loss 0.0570, acc 0.9666
step  5500, loss 0.2989, acc 0.9649
step  6000, loss 0.0503, acc 0.9722
step  6500, loss 0.2366, acc 0.9717
step  7000, loss 0.0542, acc 0.9757
step  7500, loss 0.0275, acc 0.9748
step  8000, loss 0.1192, acc 0.9754
step  8500, loss 0.0145, acc 0.9774
step  9000, loss 0.0367, acc 0.9779
step  9500, loss 0.1260, acc 0.9764
step 10000, loss 0.0411, acc 0.9798
step 10500, loss 0.0070, acc 0.9788
step 11000, loss 0.0141, acc 0.9784
step 11500, loss 0.2764, acc 0.9800
step 12000, loss 0.0341, acc 0.9803
step 12500, loss 0.0154, acc 0.9782
step 13000, loss 0.0215, acc 0.9788
step 13500, loss 0.0202, acc